# Task 1: Automatic classification of tweets that report adverse effects

In [ ]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 80 kB 7.5 MB/s 
     |████████████████████████████████| 6.6 MB 34.0 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 880 kB 20.2 MB/s 
     |████████████████████████████████| 106 kB 35.0 MB/s 
     |████████████████████████████████| 287 kB 33.3 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 


## 1. Import all the necessary libraries and data files

In [ ]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from transformers import RobertaTokenizerFast, RobertaModel
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [ ]:
train_filename = "task2_en_training.tsv"
val_filename = "task2_en_validation.tsv"

In [ ]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [ ]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (20544, 4) and validation data is (5134, 4)


In [ ]:
# Train top 5 rows
train.head().style.set_caption("Task 1: Train dataset")

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make things much worse...and why give bayer more of your money? they already screwed you once w/ essure"
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made chaos more vengeful and sadistic.
4,345567138376994816,138795534,0,"have decided to skip my #humira shot today. my body's having hysterics, need time to simmer down #rheum"


## 2. Prepare the data - Clean & Prepare for Model

In [ ]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [ ]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [ ]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [ ]:

# Train top 5 rows after pre-processing
train[['class', 'clean_tweets']].head()

,class,clean_tweets
0,0,"depression hurts , cymbalta can help"
1,0,"<user> right , but cipro can make things much worse . <repeated> and why give bayer more of your money ? they already screwed you once w / essure"
2,0,<user> are you on paxil . <repeated> i need help
3,0,<user> the lamotrigine and sjs just made chaos more vengeful and sadistic .
4,0,"have decided to skip my <hashtag> humira </hashtag> shot today . my body ' s having hysterics , need time to simmer down <hashtag> rheum </hashtag>"


In [ ]:
BATCH_SIZE = 2
N_EPOCHS = 5

In [ ]:
# Define BERT tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [ ]:
# Tokenize train and validation data
train_enc_rob = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc_rob = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [ ]:
train_enc_rob.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
train_enc_rob.input_ids.shape, train_enc_rob.attention_mask.shape

(torch.Size([20544, 128]), torch.Size([20544, 128]))

In [ ]:
def get_dataloader_rob(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [ ]:
train_dataloader_rob = get_dataloader_rob(train_enc_rob, torch.tensor(train['class'].to_list()))
valid_dataloader_rob = get_dataloader_rob(valid_enc_rob, torch.tensor(validation['class'].to_list()))

In [ ]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader_rob:
    input_ids, attn_mask, target = batch
    print(input_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([2, 128]) torch.Size([2, 128]) torch.Size([2])


## 3. Model Building - Roberta

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class ROBERTAclassifier(nn.Module):
    def __init__(self, transformer):
        super(ROBERTAclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(1024, 2)
    
    def forward(self, ip_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask)
        return  self.linear_layer(op["pooler_output"])

In [ ]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [ ]:
transformer_rob = RobertaModel.from_pretrained("roberta-large")
model_rob = ROBERTAclassifier(transformer_rob).to(device)
print(f"The model has {count_parameter(model_rob)} trainable parameters.")

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 355361794 trainable parameters.


In [ ]:
# Define optimizer 
criterion_rob = torch.nn.CrossEntropyLoss()
optim_rob = torch.optim.AdamW(model_rob.parameters(), lr = 2e-5)

In [ ]:
def train_model_rob(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, y = batch

        optim_rob.zero_grad()
        output = model(input_ids, attn_mask)
        loss = criterion_rob(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim_rob.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate_rob(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, attn_mask)
            loss = criterion_rob(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [ ]:
best_valid_loss_rob = float('inf')
total_train_loss_rob, total_valid_loss_rob = list(), list()

In [ ]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model_rob(model_rob, train_dataloader_rob)
    total_train_loss_rob.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate_rob(model_rob, valid_dataloader_rob)
    total_valid_loss_rob.append(valid_loss)

    if valid_loss < best_valid_loss_rob:
        best_valid_loss_rob = valid_loss
        best_pred, best_target = pred, target
        torch.save(model_rob.state_dict(), "model_least_loss_rob_large.pt")
        print("\nBest Model Saved!!\n")
    
    torch.save(model_rob.state_dict(), "model_checkpoint_rob_large" + str(epoch) + ".pt")
    print("Checkpoint Model Saved!\n")

    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

Streaming output truncated to the last 5000 lines.
5273it [54:52,  1.61it/s]
5274it [54:53,  1.61it/s]
5275it [54:53,  1.60it/s]
5276it [54:54,  1.60it/s]
5277it [54:55,  1.60it/s]
5278it [54:55,  1.61it/s]
5279it [54:56,  1.61it/s]
5280it [54:56,  1.61it/s]
5281it [54:57,  1.61it/s]
5282it [54:58,  1.61it/s]
5283it [54:58,  1.61it/s]
5284it [54:59,  1.61it/s]
5285it [55:00,  1.61it/s]
5286it [55:00,  1.61it/s]
5287it [55:01,  1.61it/s]
5288it [55:01,  1.61it/s]
5289it [55:02,  1.61it/s]
5290it [55:03,  1.61it/s]
5291it [55:03,  1.61it/s]
5292it [55:04,  1.61it/s]
5293it [55:05,  1.61it/s]
5294it [55:05,  1.61it/s]
5295it [55:06,  1.61it/s]
5296it [55:06,  1.61it/s]
5297it [55:07,  1.61it/s]
5298it [55:08,  1.61it/s]
5299it [55:08,  1.61it/s]
5300it [55:09,  1.61it/s]
5301it [55:09,  1.61it/s]
5302it [55:10,  1.61it/s]
5303it [55:11,  1.61it/s]
5304it [55:11,  1.61it/s]
5305it [55:12,  1.61it/s]
5306it [55:13,  1.60it/s]
5307it [55:13,  1.60it/s]
5308it [55:14,  1.60it/s]
5309it [55:14


Best Model Saved!!



 20%|██        | 1/5 [1:53:20<7:33:22, 6800.53s/it]

Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.474 | Train F1 Score: 0.000
Valid Total Loss: 0.499 | Valid F1 Score: 0.000
--------------------


Streaming output truncated to the last 5000 lines.
5274it [54:46,  1.61it/s]
5275it [54:47,  1.61it/s]
5276it [54:47,  1.61it/s]
5277it [54:48,  1.60it/s]
5278it [54:49,  1.60it/s]
5279it [54:49,  1.60it/s]
5280it [54:50,  1.61it/s]
5281it [54:50,  1.60it/s]
5282it [54:51,  1.60it/s]
5283it [54:52,  1.60it/s]
5284it [54:52,  1.61it/s]
5285it [54:53,  1.60it/s]
5286it [54:54,  1.60it/s]
5287it [54:54,  1.60it/s]
5288it [54:55,  1.60it/s]
5289it [54:55,  1.60it/s]
5290it [54:56,  1.61it/s]
5291it [54:57,  1.60it/s]
5292it [54:57,  1.60it/s]
5293it [54:58,  1.60it/s]
5294it [54:59,  1.60it/s]
5295it [54:59,  1.60it/s]
5296it [55:00,  1.60it/s]
5297it [55:00,  1.60it/s]
5298it [55:01,  1.60it/s]
5299it [55:02,  1.60it/s]
5300it [55:02,  1.61it/s]
5301it [55:03,  1.60it/s]
5302it [55:04,  1.61it/s]
5303it [55:04,  1.61it/s]
5304it [55:05,  1.60it/s]
5305it [55:05,  1.60it/s]
5306it [55:06,  1.60it/s]
5307it [55:07,  1.60it/s]
5308it [55:07,  1.60it/s]
5309it [55:08,  1.60it/s]
5310it [55:09


Best Model Saved!!



 40%|████      | 2/5 [3:46:48<5:40:14, 6804.67s/it]

Checkpoint Model Saved!

Epoch: 02
Train Total Loss: 0.466 | Train F1 Score: 0.000
Valid Total Loss: 0.484 | Valid F1 Score: 0.000
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.36it/s]
2it [00:01,  1.50it/s]
3it [00:01,  1.55it/s]
4it [00:02,  1.58it/s]
5it [00:03,  1.59it/s]
6it [00:03,  1.59it/s]
7it [00:04,  1.59it/s]
8it [00:05,  1.59it/s]
9it [00:05,  1.59it/s]
10it [00:06,  1.59it/s]
11it [00:06,  1.59it/s]
12it [00:07,  1.60it/s]
13it [00:08,  1.60it/s]
14it [00:08,  1.60it/s]
15it [00:09,  1.60it/s]
16it [00:10,  1.60it/s]
17it [00:10,  1.60it/s]
18it [00:11,  1.60it/s]
19it [00:11,  1.60it/s]
20it [00:12,  1.60it/s]
21it [00:13,  1.60it/s]
22it [00:13,  1.59it/s]
23it [00:14,  1.60it/s]
24it [00:15,  1.60it/s]
25it [00:15,  1.60it/s]
26it [00:16,  1.60it/s]
27it [00:16,  1.60it/s]
28it [00:17,  1.60it/s]
29it [00:18,  1.60it/s]
30it [00:18,  1.60it/s]
31it [00:19,  1.60it/s]
32it [00:20,  1.59it/s]
33it [00:20,  1.60it/s]
34it [00:21,  1.60it/s]
35it [00:21,  1.60it/s]
36it [00:22,  1.60it/s]
37it [00:23,  1.60it/s]
38it [00:23,  1.60it/s]
39it [00:24,  1.60it/s]
40it [00:25,  1.60it/s]
41it [00:25,  1.60it/s]
42it 

In [ ]:
print(classification_report(best_target, best_pred))

In [ ]:
!zip -r content.zip ../content/ 

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
from glob import glob

In [ ]:
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/NLP\ Final\ Project/
    time.sleep(10)

In [ ]:
# !cp -r model_least_loss_rob.pt /content/gdrive/My\ Drive/Colab\ Notebooks/NLP\ Final\ Project/

In [ ]:
# with open('/content/gdrive/My Drive/', 'w') as handle:
#     handle.write()